<a href="https://colab.research.google.com/github/Liang130520/Deep_Learning/blob/main/20221130_Deep_Learning_3_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84125825/84125825 [==============================] - 9s 0us/step


In [2]:
import glob
glob.glob("/root/.keras/datasets/aclImdb/train/*")

['/root/.keras/datasets/aclImdb/train/urls_pos.txt',
 '/root/.keras/datasets/aclImdb/train/labeledBow.feat',
 '/root/.keras/datasets/aclImdb/train/neg',
 '/root/.keras/datasets/aclImdb/train/unsupBow.feat',
 '/root/.keras/datasets/aclImdb/train/pos',
 '/root/.keras/datasets/aclImdb/train/unsup',
 '/root/.keras/datasets/aclImdb/train/urls_neg.txt',
 '/root/.keras/datasets/aclImdb/train/urls_unsup.txt']

In [3]:
import os
import glob
import pandas as pd
def getdata(mid):
    dn = os.path.dirname(dataset)
    posfn = glob.glob(os.path.join(dn, "aclImdb", mid, "pos", "*"))
    negfn = glob.glob(os.path.join(dn, "aclImdb", mid, "neg", "*"))
    contents = []
    for fn in posfn + negfn:
        with open(fn, encoding="utf-8") as f:
            contents.append(f.read())
    df = pd.DataFrame({
        "content":contents,
        "sentiment":[1] * len(posfn) + [0] * len(negfn)
    })
    return df
train_df = getdata("train")
test_df = getdata("test")

In [4]:
test_df

,content,sentiment
0,Four stories about the drug trade in Europe be...,1
1,Lloyd Hamilton was one of the most imaginative...,1
2,SPOILER WARNING<br /><br />We've all heard the...,1
3,One of the most pleasurable aspects of movie v...,1
4,I would hope so and how can I get involved?<br...,1
...,...,...
24995,"La Ragazza del Vagone Letto, or Terror Express...",0
24996,"When I saw this in the cinema, I remember winc...",0
24997,"Why Lori Petty was cast as tank girl, I'll nev...",0
24998,No emotion. Bad music (and I am a reformed eig...,0


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

layers = [
    # input:3001 -> 情緒化對象(3000種最常用的詞彙 + padding(設為0))
    # output: 情緒個數(通常100-500, 看學習問題複雜程度)
    # input_length: 一篇文章要學習多少詞彙來預測其餘資料(通常128-512)
    # 總結: 一篇文章將蒐集512個詞彙, 該詞彙皆位於3000個最常用詞彙裡, 且512中每個詞彙皆轉為100維度的向量
    # 係數個數: 3001(種詞彙) * 100(個情緒) = 300100
    Embedding(3001, 100, mask_zero=True, input_length=512),
    GlobalAveragePooling1D(),
    Dense(2, activation="softmax"),
]
model = Sequential(layers)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 512, 100)          300100    
                                                                 
 global_average_pooling1d_2   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 2)                 202       
                                                                 
Total params: 300,302
Trainable params: 300,302
Non-trainable params: 0
_________________________________________________________________


In [10]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [13]:
# tokenize: 
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["content"])
# tok.word_index # 將詞彙設定其index
# tok.index_word # 將index設定對應詞彙

In [14]:
# 文章token化: sequence
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,...,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,8,1,19,523,31,104,115,365,2665.0,2888.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,312,1807,755,53,3,369,36,1,308.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,3,348,20,459,4,48,386,99.0,382.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1230,215,96,562,3,678,15,1,11.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,432,2,216,2,171,635,42,1.0,330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,440,1947,13,239,1,88,4,29,55.0,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,11,6,1085,784,120,41,28,974,1.0,974.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,11,17,66,3,897,672,4,706,297.0,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,171,36,1,2592,48,123,572,5,1.0,84.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# padding(擴充) and truncating
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=512)
x_test_pad = pad_sequences(x_test_seq, maxlen=512)
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,...,1,1664,10,59,404,199,11,19,104,53
1,0,0,0,0,0,0,0,0,0,0,...,440,2095,10,437,11,1519,256,1992,43,1
2,1984,33,89,356,98,170,4,1,540,1,...,2374,634,1226,5,1,245,1085,43,4,155
3,0,0,0,0,0,0,0,0,0,0,...,215,11,19,20,285,9,1011,5,409,1433
4,0,0,0,0,0,0,0,0,0,0,...,871,553,41,36,1,42,443,2,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,...,1056,146,45,22,178,5,291,43,339,155
24996,0,0,0,0,0,0,0,0,0,0,...,10,188,819,43,135,10,89,456,41,95
24997,0,0,0,0,0,0,0,0,0,0,...,17,77,127,53,8,1,348,17,1049,512
24998,0,0,0,0,0,0,0,0,0,0,...,1006,21,3,75,194,18,54,437,15,1019


In [18]:
import numpy as np
y_train = np.array(train_df["sentiment"])
y_test = np.array(test_df["sentiment"])
y_train

array([1, 1, 1, ..., 0, 0, 0])

In [19]:
# batch_size: 我看多少筆, 把所有筆的斜率做一個平均, 再調整
# 圖片小(不吃那麼多ram):100-200 圖片大(吃很多ram):10-20
# 60000筆, 200比一次調整  -> 300次調整
# epochs: 整份資料看幾遍(你覺得模型訓練好就停下來)
# validation_split: 切出一部份資料來做模型確認
# epochs + validation: 什麼時候停下來(val_loss平滑的時候就停下來)
# epoch: 60000筆(90%) -> 54000 / 200 -> 270調整
# verbose: 決定log印多少 1(default):進度條 2:沒有進度條, 0:完全不印
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
callbacks = [
    ModelCheckpoint("sentiment.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad,
     y_train,
     batch_size=200,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks,
     verbose=2)

Epoch 1/50
113/113 - 4s - loss: 0.6394 - accuracy: 0.6349 - val_loss: 0.6470 - val_accuracy: 0.6352 - 4s/epoch - 32ms/step
Epoch 2/50
113/113 - 1s - loss: 0.4875 - accuracy: 0.8182 - val_loss: 0.4785 - val_accuracy: 0.7940 - 593ms/epoch - 5ms/step
Epoch 3/50
113/113 - 1s - loss: 0.3809 - accuracy: 0.8594 - val_loss: 0.3849 - val_accuracy: 0.8440 - 584ms/epoch - 5ms/step
Epoch 4/50
113/113 - 1s - loss: 0.3265 - accuracy: 0.8784 - val_loss: 0.4040 - val_accuracy: 0.8236 - 545ms/epoch - 5ms/step
Epoch 5/50
113/113 - 1s - loss: 0.2946 - accuracy: 0.8884 - val_loss: 0.3653 - val_accuracy: 0.8456 - 572ms/epoch - 5ms/step
Epoch 6/50
113/113 - 1s - loss: 0.2734 - accuracy: 0.8953 - val_loss: 0.3511 - val_accuracy: 0.8504 - 587ms/epoch - 5ms/step
Epoch 7/50
113/113 - 1s - loss: 0.2578 - accuracy: 0.9016 - val_loss: 0.3366 - val_accuracy: 0.8556 - 568ms/epoch - 5ms/step
Epoch 8/50
113/113 - 1s - loss: 0.2462 - accuracy: 0.9067 - val_loss: 0.3256 - val_accuracy: 0.8668 - 598ms/epoch - 5ms/step
Ep

In [20]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 3ms/step - loss: 0.2891 - accuracy: 0.8819


[0.28912800550460815, 0.8819199800491333]